# Prep Glider Data

## Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
import xarray as xr
import gsw
import dbdreader
import slocum_ad2cp


## Some functions

In [3]:
########################################################################################################
def load_glider_data(data_files, cac_dir):
    """
    Inspired by code in: https://github.com/truedichotomy/CTD_thermal_lag/
    
    Prepare data for analysis. Reads specific sensors into a dataframe and interpolates glider sensors onto science time.
    Assigns profile id to each value, removes invalid values, and converts units of specific variables.

    Args:
        data_files (string): Pathname to desired .EBD and .DBD files for a deployment
        cac_dir (string): Pathname to directory housing associated cache files for a deployment

    Returns:
        Pandas dataframe (group) with glider data
    """
    ## Initialize dbd class
    dbd = dbdreader.MultiDBD(pattern = data_files,
                             cacheDir=cac_dir)
    
    ## Since time stamp of flight and science parameters on CTD time stamp
    tctd, C, T, P, buoyancy_change, pitch, lat, lon, num_inflections = dbd.get_CTD_sync("m_de_oil_vol", "m_pitch","m_lat","m_lon","m_tot_num_inflections")
    ## num inflections is weird
    num_inflections = num_inflections.astype(int)

    
    ## Stuff data in a pandas dataframe
    d = {'ctd_time': tctd, 'conductivity': C, 'temperature':T, 'pressure':P, 'latitude':lat, 'longitude':lon, 'num_inflections':num_inflections}
    sci_data = pd.DataFrame(d)
    
    ## Now a lot of conversions
    ## Format time
    sci_data['time'] = pd.to_datetime(sci_data['ctd_time'], unit='s')
    ## Convert pressure to dbar
    sci_data['pressure'] = sci_data['pressure']*10
    ## Convert from pressure to depth
    sci_data['depth'] = -gsw.z_from_p(sci_data.pressure, sci_data.latitude)
    ## Convert to practical salinity
    sci_data['practical_salinity'] = gsw.SP_from_C(sci_data.conductivity*10, sci_data.temperature, sci_data.pressure) ## C in S/m, T is in-situ, P in dbar 
    ## Convert to absolute salinity
    sci_data['absolute_salinity'] = gsw.SA_from_SP(sci_data.practical_salinity, sci_data.pressure, sci_data.longitude, sci_data.latitude)
    ## Convert to potential temperature ref to 0 meters
    sci_data['potential_temperature'] = gsw.pt0_from_t(sci_data.absolute_salinity, sci_data.temperature, sci_data.pressure)
    ## Convert to conservative temperature
    sci_data['conservative_temperature'] = gsw.CT_from_t(sci_data.absolute_salinity, sci_data.temperature, sci_data.pressure)
    ## in situ density
    sci_data['rho'] = gsw.rho_t_exact(sci_data.absolute_salinity, sci_data.temperature, sci_data.pressure) ## T is in-situ
    ## potential density
    sci_data['sigma'] = gsw.sigma0(sci_data.absolute_salinity, sci_data.conservative_temperature)+1000
    
    ## Assign profiles ids based on m_tot_num_inflections
    ## Forward and backfill m_tot_num_inflections variable so every value has an associated correct inflection count 
    sci_data['num_inflections'].ffill(inplace=True)
    sci_data['num_inflections'].bfill(inplace=True)
    
    # Drop all invalid and duplicate ctd timestamps and invalid lat/lon values
    sci_data = sci_data[sci_data['ctd_time'].ne(pd.Timestamp('1970-01-01 00:00:00.00'))].dropna(subset=['ctd_time']) 
    sci_data = sci_data.drop_duplicates(subset=['ctd_time'])
    sci_data = sci_data[sci_data['latitude'].ne(0)].dropna(subset=['latitude']) ## .ne() not equal
    sci_data = sci_data[sci_data['longitude'].ne(0)].dropna(subset=['longitude'])
    sci_data = sci_data.dropna(subset=['pressure'])
    
    #Create profile_id column based on when the m_tot_num_inflections variable iterates
    sci_data['profile_id'] = sci_data.groupby('num_inflections').ngroup()
    sci_data = sci_data.sort_values(by='time')
    sci_data.reset_index(drop=True,inplace=True) #reset indices
    
    return sci_data


########################################################################################################

def depth_bin_average(df, dz=5):
    """
    Function to bin-average data by depth and construct an xarray dataset dynamically.

    Parameters:
        df (pd.DataFrame): DataFrame with columns ['time', 'depth', 'latitude', 'longitude', ...]
        dz (int, optional): Depth bin size.

    Returns:
        xr.Dataset: Dataset with depth-binned profiles along the time axis.
    """

    ds = None  # Initialize an empty dataset

    df = df.reset_index(drop=True)

    # Define depth bins
    max_depth = df["depth"].max()
    depth_bins = np.arange(np.floor(0 / dz) * dz, np.ceil(max_depth / dz) * dz + dz, dz)
    depth_mids = depth_bins[:-1] + dz / 2  # Midpoint of bins
    
    for prof in df["profile_id"].unique():
        profile = df[df["profile_id"] == prof]

        # Dictionary to hold bin-averaged variables
        data_vars = {}

        # Bin-average each variable (excluding non-data columns)
        for var in profile.columns:
            if var in ["time", "latitude", "longitude", "depth", "profile_id","pressure",'num_inflections']:
                continue  # Skip non-data variables
            else:
                # Bin the data
                binned_means = np.array([
                    np.nanmean(profile[(profile["depth"] >= depth_bins[i]) & (profile["depth"] < depth_bins[i+1])][var])
                    for i in range(len(depth_bins) - 1)
                ])
                # Count number of non-nan in bins
                binned_counts = np.array([
                    np.count_nonzero(~np.isnan(profile[(profile["depth"] >= depth_bins[i]) & (profile["depth"] < depth_bins[i+1])][var]))
                    for i in range(len(depth_bins) - 1)
                ])
                
                # Handle cases where no data exists in a bin
                if np.isnan(binned_means).all():
                    data_vars[var+"_counts"] = (["depth"], np.repeat(np.nan,len(depth_mids)))
                    
                # Store in dictionary
                data_vars[var] = (["depth"], binned_means)
                data_vars[var+"_counts"] = (["depth"], binned_counts)        

                    
        ## Deal with time, latitude, and longitude
        if profile.depth.iloc[0] > 10: ## if first non-nan depth is deeper than 10 m, fill with profile END values
            prof_time = profile.time.iloc[-1]
            prof_lat = profile.latitude.iloc[-1]
            prof_lon = profile.longitude.iloc[-1]
        elif profile.depth.iloc[0] < 10: ## if first depth is shallower than 10 m
            prof_time = profile.time.iloc[0]
            prof_lat = profile.latitude.iloc[0]
            prof_lon = profile.longitude.iloc[0]

        
        # Create a dataset for this profile
        profile_ds = xr.Dataset(
            data_vars,
            coords={"depth": ("depth", depth_mids), "time": [pd.to_datetime(prof_time)]}
        )
        
        # Add latitude and longitude as coordinates (keeping them 1D for alignment)
        profile_ds = profile_ds.assign_coords(
            latitude=("time", [prof_lat]),
            longitude=("time", [prof_lon])
        )

        # Merge into the master dataset
        if ds is None:
            ds = profile_ds  # First profile initializes the dataset
        else:
            ds = xr.concat([ds, profile_ds], dim="time")  # Append new profile along time axis

    ## make sure time is in correct format
    ds['time'] = pd.to_datetime(ds.time)
    # Sort the dataset by the 'time' coordinate
    ds = ds.sortby('time')
    ## Flip
    ds = ds.transpose()

    return ds



########################################################################################################

def depth_bin_average_erddap(df, dz=5):
    """
    Function to bin-average data by depth and construct an xarray dataset dynamically.

    Parameters:
        df (pd.DataFrame): DataFrame with columns ['time', 'depth', 'latitude', 'longitude', ...]
        dz (int, optional): Depth bin size.

    Returns:
        xr.Dataset: Dataset with depth-binned profiles along the time axis.
    """

    ds = None  # Initialize an empty dataset

    df = df.reset_index(drop=True)

    # Define depth bins
    max_depth = df["depth"].max()
    depth_bins = np.arange(np.floor(0 / dz) * dz, np.ceil(max_depth / dz) * dz + dz, dz)
    depth_mids = depth_bins[:-1] + dz / 2  # Midpoint of bins
    
    for prof in df["profile_id"].unique():
        profile = df[df["profile_id"] == prof]

        # Dictionary to hold bin-averaged variables
        data_vars = {}

        # Bin-average each variable (excluding non-data columns)
        for var in profile.columns:
            if var in ["time", "latitude", "longitude", "depth", 'depth_interpolated',"profile_id","pressure",'num_inflections']:
                continue
                # Skip non-data variables
            
            #if var in ['oxygen_concentration_shifted_mgL']:
            # Bin the data
            binned_means = np.array([
                np.nanmean(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var])
                for i in range(len(depth_bins) - 1)
            ])
            # Count number of non-nan in bins
            
            binned_counts = np.array([
                np.count_nonzero(~np.isnan(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var]))
                for i in range(len(depth_bins) - 1)
            ])
            
            # Handle cases where no data exists in a bin
            if np.isnan(binned_means).all():
                data_vars[var+"_counts"] = (["depth"], np.repeat(np.nan,len(depth_mids)))
                
            # Store in dictionary
            data_vars[var] = (["depth"], binned_means)
            data_vars[var+"_counts"] = (["depth"], binned_counts)
            
        ## Deal with time, latitude, and longitude
        if profile.depth_interpolated.iloc[0] > 10: ## if first non-nan depth is deeper than 10 m, fill with profile END values
            prof_time = profile.time.iloc[-1]
            prof_lat = profile.latitude.iloc[-1]
            prof_lon = profile.longitude.iloc[-1]
        elif profile.depth_interpolated.iloc[0] < 10: ## if first depth is shallower than 10 m
            prof_time = profile.time.iloc[0]
            prof_lat = profile.latitude.iloc[0]
            prof_lon = profile.longitude.iloc[0]

        
        # Create a dataset for this profile
        profile_ds = xr.Dataset(
            data_vars,
            coords={"depth": ("depth", depth_mids), "time": [pd.to_datetime(prof_time)]}
        )
        
        # Add latitude and longitude as coordinates (keeping them 1D for alignment)
        profile_ds = profile_ds.assign_coords(
            latitude=("time", [prof_lat]),
            longitude=("time", [prof_lon])
        )

        # Merge into the master dataset
        if ds is None:
            ds = profile_ds  # First profile initializes the dataset
        else:
            ds = xr.concat([ds, profile_ds], dim="time")  # Append new profile along time axis

    ## make sure time is in correct format
    ds['time'] = pd.to_datetime(ds.time)
    # Sort the dataset by the 'time' coordinate
    ds = ds.sortby('time')
    ## Flip
    ds = ds.transpose()

    return ds


## Load glider data

In [4]:
data_files = "../data/logs/*.[DE]BD"
cac_dir = "../data/cache/"
gdf = load_glider_data(data_files, cac_dir)
gdf

/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_40268/4242905231.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sci_data['num_inflections'].ffill(inplace=True)
/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_40268/4242905231.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

,ctd_time,conductivity,temperature,pressure,latitude,longitude,num_inflections,time,depth,practical_salinity,absolute_salinity,potential_temperature,conservative_temperature,rho,sigma,profile_id
0,1.713537e+09,5.84874,28.553400,0.02,17.779364,-67.057679,34482,2024-04-19 14:23:50.975585938,0.019880,36.164124,36.335056,28.553395,28.511694,1023.088162,1023.087826,0
1,1.713537e+09,5.83814,28.491800,0.10,17.779366,-67.057692,34482,2024-04-19 14:23:52.975677490,0.099402,36.137071,36.307875,28.491776,28.451158,1023.088658,1023.087980,0
2,1.713537e+09,5.83715,28.476801,0.01,17.779368,-67.057704,34482,2024-04-19 14:23:54.979285717,0.009940,36.141626,36.312452,28.476799,28.435976,1023.096680,1023.096379,0
3,1.713537e+09,5.83656,28.463600,0.07,17.779369,-67.057717,34482,2024-04-19 14:23:56.975952148,0.069581,36.147559,36.318412,28.463583,28.422501,1023.105780,1023.105227,0
4,1.713537e+09,5.83510,28.455299,0.08,17.779371,-67.057730,34482,2024-04-19 14:23:58.979560375,0.079521,36.143689,36.314524,28.455280,28.414353,1023.105671,1023.105074,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940378,1.721734e+09,5.91612,29.555599,0.05,17.850085,-67.036728,35840,2024-07-23 11:25:15.918731689,0.049701,35.873666,36.043224,29.555587,29.527345,1022.533187,1022.532792,1350
3940379,1.721734e+09,5.91661,29.560499,0.02,17.850085,-67.036729,35840,2024-07-23 11:25:17.924262524,0.019880,35.873370,36.042927,29.560494,29.532271,1022.531172,1022.530904,1350
3940380,1.721734e+09,5.91662,29.557100,0.04,17.850085,-67.036731,35840,2024-07-23 11:25:19.919921875,0.039761,35.875970,36.045539,29.557090,29.528751,1022.534362,1022.534010,1350
3940381,1.721734e+09,5.91683,29.559900,0.12,17.850086,-67.036732,35840,2024-07-23 11:25:21.918701172,0.119282,35.875296,36.044862,29.559871,29.531564,1022.533247,1022.532560,1350


## Also load an ERDAPP df of glider data

In [5]:
ds_id = 'ru29-20240419T1430-profile-sci-delayed'

## Load flight data
variables = ['depth', 'depth_interpolated', 'pressure', 'latitude', 'longitude', 'time', 'temperature', 'salinity','profile_id', 'oxygen_concentration_shifted_mgL']
egdf = slocum_ad2cp.get_erddap_dataset(ds_id, server='http://slocum-data.marine.rutgers.edu/erddap', variables = variables, filetype='dataframe')
egdf.columns = variables
egdf = egdf.rename(columns={'salinity': 'practical_salinity'})
egdf['absolute_salinity'] = gsw.SA_from_SP(egdf.practical_salinity, egdf.pressure, egdf.longitude, egdf.latitude)
## Convert to potential temperature ref to 0 meters
egdf['potential_temperature'] = gsw.pt0_from_t(egdf.absolute_salinity, egdf.temperature, egdf.pressure)
## Convert to conservative temperature
egdf['conservative_temperature'] = gsw.CT_from_t(egdf.absolute_salinity, egdf.temperature, egdf.pressure)
## in situ density
egdf['rho'] = gsw.rho_t_exact(egdf.absolute_salinity, egdf.temperature, egdf.pressure) ## T is in-situ
## potential density
egdf['sigma'] = gsw.sigma0(egdf.absolute_salinity, egdf.conservative_temperature)+1000
## Reformate time
egdf['time'] = pd.to_datetime(egdf.time)
egdf

,depth,depth_interpolated,pressure,latitude,longitude,time,temperature,practical_salinity,profile_id,oxygen_concentration_shifted_mgL,absolute_salinity,potential_temperature,conservative_temperature,rho,sigma
0,-0.019880,0.119282,-0.02,17.779417,-67.057943,2024-04-19 14:32:05+00:00,28.7743,36.168125,1.713537e+09,6.66336,36.339076,28.774305,28.732625,1023.017306,1023.017154
1,NaN,0.119282,NaN,17.779425,-67.057943,2024-04-19 14:32:07+00:00,NaN,NaN,1.713537e+09,NaN,NaN,NaN,NaN,NaN,NaN
2,0.119282,0.119282,0.12,17.779429,-67.057943,2024-04-19 14:32:08+00:00,28.8068,36.144070,1.713537e+09,NaN,36.314907,28.806771,28.766137,1022.988962,1022.988225
3,NaN,0.084492,NaN,17.779430,-67.057943,2024-04-19 14:32:08+00:00,NaN,NaN,1.713537e+09,NaN,NaN,NaN,NaN,NaN,NaN
4,0.049701,0.049701,0.05,17.779438,-67.057942,2024-04-19 14:32:10+00:00,29.0814,36.183212,1.713537e+09,6.66912,36.354234,29.081388,29.039338,1022.925897,1022.925475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7268920,NaN,0.096088,NaN,17.847839,-67.038801,2024-07-23 11:25:35+00:00,NaN,NaN,1.721733e+09,NaN,NaN,NaN,NaN,NaN,NaN
7268921,0.119282,0.119282,0.12,17.847840,-67.038804,2024-07-23 11:25:36+00:00,29.5625,35.876640,1.721733e+09,NaN,36.046212,29.562470,29.534109,1022.533372,1022.532685
7268922,0.129222,0.129222,0.13,17.847843,-67.038811,2024-07-23 11:25:38+00:00,29.5619,35.873867,1.721733e+09,NaN,36.043426,29.561868,29.533626,1022.531538,1022.530810
7268923,NaN,0.064611,NaN,17.847843,-67.038813,2024-07-23 11:25:38+00:00,NaN,NaN,1.721733e+09,NaN,NaN,NaN,NaN,NaN,NaN


## Depth grid glider data
### These take ~25 minutes each

In [5]:
gridded_glider = depth_bin_average(gdf,dz=2)
gridded_glider

/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_72386/1288933893.py:106: RuntimeWarning: Mean of empty slice
  np.nanmean(profile[(profile["depth"] >= depth_bins[i]) & (profile["depth"] < depth_bins[i+1])][var])


<xarray.Dataset> Size: 97MB
Dimensions:                          (time: 1351, depth: 499)
Coordinates:
  * depth                            (depth) float64 4kB 1.0 3.0 ... 995.0 997.0
  * time                             (time) datetime64[ns] 11kB 2024-04-19T14...
    latitude                         (time) float64 11kB 17.78 17.78 ... 17.85
    longitude                        (time) float64 11kB -67.06 ... -67.04
Data variables: (12/18)
    ctd_time                         (depth, time) float64 5MB 1.714e+09 ... nan
    ctd_time_counts                  (depth, time) int64 5MB 65 0 79 0 ... 0 0 0
    conductivity                     (depth, time) float64 5MB 5.855 nan ... nan
    conductivity_counts              (depth, time) int64 5MB 65 0 79 0 ... 0 0 0
    temperature                      (depth, time) float64 5MB 28.63 nan ... nan
    temperature_counts               (depth, time) int64 5MB 65 0 79 0 ... 0 0 0
    ...                               ...
    conservative_temperature         (depth, time) float64 5MB 28.59 nan ... nan
    conservative_temperature_counts  (depth, time) int64 5MB 65 0 79 0 ... 0 0 0
    rho                              (depth, time) float64 5MB 1.023e+03 ... nan
    rho_counts                       (depth, time) int64 5MB 65 0 79 0 ... 0 0 0
    sigma                            (depth, time) float64 5MB 1.023e+03 ... nan
    sigma_counts                     (depth, time) int64 5MB 65 0 79 0 ... 0 0 0

In [6]:
gridded_glider_e = depth_bin_average_erddap(egdf,dz=2)
gridded_glider_e

/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_72386/1288933893.py:202: RuntimeWarning: Mean of empty slice
  np.nanmean(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var])
/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_72386/1288933893.py:202: RuntimeWarning: Mean of empty slice
  np.nanmean(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var])
/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_72386/1288933893.py:202: RuntimeWarning: Mean of empty slice
  np.nanmean(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var])
/var/folders/m9/d8bbz1wx4f50q99kfb1jv9pw0000gn/T/ipykernel_72386/1288933893.py:202: RuntimeWarning: Mean of empty slice
  np.nanmean(profile[(profile["depth_interpolated"] >= depth_bins[i]) & (profile["depth_interpolated"] < depth_bins[i+1])][var])
/var

<xarray.Dataset> Size: 87MB
Dimensions:                                  (time: 1362, depth: 499)
Coordinates:
  * depth                                    (depth) float64 4kB 1.0 ... 997.0
  * time                                     (time) datetime64[ns] 11kB 2024-...
    latitude                                 (time) float64 11kB 17.78 ... 17.85
    longitude                                (time) float64 11kB -67.06 ... -...
Data variables: (12/16)
    temperature                              (depth, time) float64 5MB 28.88 ...
    temperature_counts                       (depth, time) int64 5MB 31 21 ... 0
    practical_salinity                       (depth, time) float64 5MB 36.16 ...
    practical_salinity_counts                (depth, time) int64 5MB 31 21 ... 0
    oxygen_concentration_shifted_mgL         (depth, time) float64 5MB 6.62 ....
    oxygen_concentration_shifted_mgL_counts  (depth, time) int64 5MB 27 10 ... 0
    ...                                       ...
    conservative_temperature                 (depth, time) float64 5MB 28.84 ...
    conservative_temperature_counts          (depth, time) int64 5MB 31 21 ... 0
    rho                                      (depth, time) float64 5MB 1.023e...
    rho_counts                               (depth, time) int64 5MB 31 21 ... 0
    sigma                                    (depth, time) float64 5MB 1.023e...
    sigma_counts                             (depth, time) int64 5MB 31 21 ... 0

## Save gridded output

In [7]:
gridded_glider.to_netcdf('../data/processed_nc/RU29_2024_depth_gridded_glider.nc')

In [8]:
gridded_glider_e.to_netcdf('../data/processed_nc/RU29_2024_depth_gridded_glider_erddap_oxygen.nc')